In [2]:
import pandas as pd
import numpy as np
import os
import sys

In [3]:
# Read raw data
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data.read_data import load_and_label_raw_data, replace_categorical_labels
raw_path = "../data/raw/german/german.data"
processed_path = "../data/processed/processed_data.csv"
df = load_and_label_raw_data(raw_path)
df_label = replace_categorical_labels(df)

In [ ]:
# Feature Groups
# From EDA: Treat these as categorical (though they are int)
categorical_int_features = [
    'installment_rate', 'residence_since', 'existing_credits', 'num_liable_people'
]

numeric_features = [
    'duration_mon', 'credit_amount', 'age'
]

categorical_object_features = [
    'account_status', 'credit_history', 'purpose', 'savings', 'employment_yr',
    'personal_status_sex', 'other_debtors', 'property', 'other_installment_plans',
    'housing', 'job', 'telephone', 'foreign_worker'
]

categorical_features = categorical_object_features + categorical_int_features




,duration_mon,credit_amount,installment_rate,residence_since,age,existing_credits,num_liable_people,account_status,credit_history,purpose,...,employment_yr,personal_status_sex,other_debtors,property,other_installment_plans,housing,job,telephone,foreign_worker,target
0,6,1169,4,4,67,2,1,< 0 DM,critical account,radio/TV,...,>= 7 years,male-single,none,real estate,none,own,skilled,yes,yes,1
1,48,5951,2,2,22,1,1,0 <= ... < 200 DM,existing credits paid,radio/TV,...,1-4 years,female-married,none,real estate,none,own,skilled,none,yes,0
2,12,2096,2,3,49,1,2,no checking account,critical account,education,...,4-7 years,male-single,none,real estate,none,own,unskilled-resident,none,yes,1
3,42,7882,2,4,45,1,2,< 0 DM,existing credits paid,furniture,...,4-7 years,male-single,guarantor,life insurance,none,for free,skilled,none,yes,1
4,24,4870,3,4,53,2,2,< 0 DM,delayed payments,car (new),...,1-4 years,male-single,none,unknown,none,for free,skilled,none,yes,0
